In [29]:
## Instalando a biblioteca 
#pip install pynliner

In [ ]:
#Importando bibliotecas que iremos utilizar
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [ ]:
#Criando função de busca
def search(soup, tag, identifier, desc):
    event_found = soup.findAll(tag, {identifier: desc})
    return event_found

In [ ]:
#Criando Função de retorno de conexão
def get_soup(url):
    html = requests.get(url)

    if html.status_code != 200:
        print(">> Falha na requisição! <<")
    else:
        html_content = html.content

    soup = BeautifulSoup(html_content, 'html.parser')

    return soup

In [ ]:
# Escopo e estrutura da operação de raspagem realizada
def format_text(element):
    text = element.text.title()
    return text

def format_oxvalues(data):
    return str(data/100).replace('.', ',')

events = {}
titles = []
dates = []

url = "https://www.cepea.esalq.usp.br/br/indicador/boi-gordo.aspx"

soup = get_soup(url)
found_table = search(soup, "table", "id", "imagenet-indicador1")

data_frame = pd.read_html(str(found_table))[0]

#Data treatment
(data_frame['Valor R$*']) = [format_oxvalues(value) for value in (data_frame['Valor R$*'])]
(data_frame['Valor US$*']) = [format_oxvalues(value) for value in (data_frame['Valor US$*'])]
data_frame = data_frame.rename(columns = {'Unnamed: 0': 'Data'}, inplace = False)

#Final value
ox_table = data_frame.loc[0]



In [ ]:
from pynliner import Pynliner

css = """\
      * {
          font-family: Arial, Helvetica, sans-serif;
      }
      #fat_ox {
          padding-left: 7%;
          width: 55%;
          height: 65%;
      }
      #ox_table {
          border-collapse: collapse;
          width: 50%;
      }
      #ox_table td, #ox_table th {
          border: 1px solid #ddd;
          padding: 7px;
      }
      #ox_table th {
        text-align: left;
        background-color: #4CAF50;
        color: white;
      }
    """

html = """\
<html>
  <head>
    <meta content="text/html;charset=utf-8" http-equiv="Content-Type">
    <meta content="utf-8" http-equiv="encoding">
  </head>
  <body>
    <img id="globo_rural" src="http://3.bp.blogspot.com/-XPALlNX7vis/UjT9CI-j8SI/AAAAAAAAADs/CtyTDmMYj00/s640/20100817112517-globo_rural.jpg" alt="Globo Rural.jpg" >
    <p>
      Saudações, segue a cotação mais atual do arroba do boi gordo. 
      <h3>Boi Gordo</h3>
      Arroba (15kg) - à vista<br>
      <br>
      <div id="table_img">
        <table id="ox_table">
          <tr>
            <th>Data</th>
            <th>Valor R$</th>
            <th>Var. /Dia</th>
            <th>Var. /Mês</th>
            <th>Valor US$</th>
          </tr>
          <tr>
            <td>{data}</td>
            <td>{valor_rs}</td>
            <td>{var_dia}</td>
            <td>{var_mes}</td>
            <td>{valor_us}</td>
          </tr>
        </table>
        <br>
        Fonte: CEPEA
        </p>
        <br>
        <br>
        <img id="fat_ox" src="https://blogs.canalrural.com.br/blogdoscot/wp-content/uploads/sites/7/2018/11/181114_Imagem_Carta_Boi_2_nm.jpg" alt="Boi Gordo.jpg"  >
      </div>
  </body>
</html>
""".format(data=ox_table[0], valor_rs=ox_table[1], var_dia=ox_table[2], var_mes=ox_table[3], valor_us=ox_table[4])

p = Pynliner()

#Put the css string into html string
body =  p.from_string(html).with_cssString(css).run()

In [ ]:
#Gerando a data de atualização do script para preencher no salvamento do CSV
from datetime import datetime
data_atual = datetime.today()
data_em_texto = data_atual.strftime('%d%m%Y')
data_em_texto

'11052021'

In [ ]:
# Gerando um DF com base no HTML construido
data_frame = pd.read_html(str(body))[0]
data_frame.head()

,Data,Valor R$,Var. /Dia,Var. /Mês,Valor US$
0,10/05/2021,3114,"-0,02%","-0,50%",5953


In [ ]:
#Validando pasta que iremos executar/salvar o CSV
import os
os.chdir("/content/drive/My Drive/arroba_boi")
!ls

body.py  @boi.ipynb  cepea.py  getsoup.py  search_event.py  send.py


In [ ]:
#salvar arquivo.csv"
data_frame.to_csv('/content/drive/My Drive/arroba_boi/BOI_GORDO_'+data_em_texto+'.csv',index=False)
